# Символьное дифференцирование

## Порядок сдачи домашнего

Вам требуется создать гит репозиторий куда вы будете складывать все ваши домашние. Под каждое домашнее вы создаете отдельную ветку куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего) или присылаете код в СДО. Ревьювером назначаете http://github.com/michael15346/ и https://github.com/shgpavel . Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 18 ноября 10:00**

Символьное дифференцирование это инструмент для автоматического вывода формул производных, который открывает возможности для анализа сложных функций, оптимизации процессов и работы с уравнениями. Мы уже на многих занятиях сталкивались с этой темой - давайте попробуем реализовать собственное!

## Выражение

Создадим основной класс `Expr`, от которого будут наследоваться различные типы выражений, такие как константы, переменные, суммы, произведения и другие. Класс должен содержать методы:
* `__call__`, который будет вычислять значение выражения, используя переданный ему контекст (словарь, связывающий имена переменных с их значениями).
* `d`, принимающий имя переменной, по которой требуется вычислить производную, и возвращающий выражение, представляющее производную по этой переменной.

Эти методы нужно будет переопределить в каждом из подклассов для корректного выполнения операций.

In [136]:
class Expr:
    def __call__(self, **context):
        pass
    
    def d(self, wrt):
        pass

    def __pos__(self):
        return Const(1)*self

    def __neg__(self):
        return Const(-1)*self

    def __add__(self, other):
        return Sum(self, other)
    
    def __sub__(self, other):
        return Sum(self, Const(-1) * other)
    
    def __mul__(self, other):
        return Product(self, other)
    
    def __truediv__(self, other):
        return Fraction(self, other)

    

Создайте классы для двух видов выражений: `Const`, представляющий константу, и` Var`, представляющий переменную. Чтобы упростить использование, вместо обращения к конструкторам этих классов, будем использовать их однобуквенные сокращённые обозначения.

**Пример использования:**
```python
V = Var
C = Const

C(5)()
5
C(5).d(V("x"))()
0
V("x")(x=5)
5
V("x").d(V("y"))(x=5)
0
V("x").d(V("x"))(x=5)
1
```

In [137]:
class Var(Expr):
    def __init__(self, var):
        self.var = var

    def __call__(self, **context):
        return context[self.var]

    def d(self, wrt):
        if self.var == wrt.var:
            return Const(1)
        return Const(0)
 
class Const(Expr):
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        return Const(0)


V = Var
C = Const

## Бинарные операции

Создайте классы для бинарных операций: `Sum`, `Product` и `Fraction`. Поскольку бинарные операции определяются двумя операндами, их конструктор будет одинаковым для всех этих классов. Поэтому его можно вынести в отдельный базовый класс, чтобы избежать дублирования кода.

In [138]:
class BinOp(Expr):
    def __init__(self, expr1, expr2):
        self.expr1, self.expr2 = expr1, expr2

Реализуйте `Sum` для суммирования, `Product` для умножения и `Fraction` для деления.

**Пример использования:**

```python
Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5)
7.0
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2)
3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2)
-3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2)
-1.25
```

In [139]:
class Sum(BinOp):
    def d(self, wrt):
        return self.expr1.d(wrt) + self.expr2.d(wrt)
        
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)
    

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        u = self.expr1
        du = self.expr1.d(wrt)
        v = self.expr2
        dv = self.expr2.d(wrt)

        return u*dv + v*du

class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        u = self.expr1
        du = self.expr1.d(wrt)
        v = self.expr2
        dv = self.expr2.d(wrt)

        return (v*du - u*dv) / (v * v)

In [140]:
(V("y") + V("x")).d(V("x"))(x = 5)

1

In [141]:
(V("y") + V("x")*V("x")).d(V("x"))(x = 5)

10

In [142]:
(V("y") + V("y") / V("x")).d(V("x"))(x = 5, y = 3)

-0.12

In [143]:
-V("x")(x = 2)

-2

## Перегрузка операторов

Добавьте перегрузку операторов в базовых класс `Expr`. Обратите что в классах мы можем тоже заменить на использование операторов.
```python  
-e         e.__neg__()
+e         e.__pos__()
e1 + e2    e1.__add__(e2)
e1 - e2    e1.__sub__(e2)
e1 * e2    e1.__mul__(e2)
e1 / e2    e1.__truediv__(e2)
```

**Пример использования:**

```python
(V("x") * V("x") / V("y"))(x=5, y=2.5)
10.0
```

## Метод Ньютона-Рафсона

Напишите функцию `newton_raphson`, которая принимает дифференцируемую функцию  $f$  от переменной  $x$ , начальное приближение  $x_0$ , и положительное число  $\epsilon$ , задающее точность вычислений. Функция должна возвращать значение  $x$ , при котором  $f(x)$  становится равным нулю. Метод Ньютона-Рафсона выполняет итеративный поиск корня функции  $f(x)$ , начиная с начального значения  $x_0$ , и использует правило  
$$x_{n+1} = x_n - \frac{f(x_n)}{f{\prime}(x_n)}$$  
для обновления  $x$  на каждом шаге. Итерации продолжаются до тех пор, пока условие остановки  $|x_{n+1} - x_n| \leq \epsilon$  не будет выполнено.

**Пример использования:**

```python
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
zero, f(x=zero)
(1.000000000001132, -2.490496697760136e-11)
```

In [144]:
def newton_raphson(expr, x0, eps=1e-4):
    x = V("x")
    prev = x0
    xn = x0 + 2*eps
    q = x - (expr / expr.d(x))

    while abs(xn - prev) > eps:
        prev = xn
        xn = q(x=xn)

    return xn

In [145]:
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
zero, f(x=zero)

(1.000000000001132, -2.490496697760136e-11)

## ТЕСТЫ

In [146]:
x = Var("x")
y = Var("y")
z = Var("z")

- Сложение

In [147]:
def test_add_standard1(x, y):
    assert (x + y)(x = 1, y = 1) == 2 #test_add_standard1

def test_add_standard2(x, y):
    assert (x + y)(x = 2, y = 5) == 7 #test_add_standard2

def test_add_zeros(x, y):
    assert (x + y)(x = 0, y = 0) == 0 #test_add_zeros

def test_add_negneg(x, y):
    assert (x + y)(x = -1, y = -1) == -2 #test_add_negneg

def test_add_negpos(x, y):
    assert (x + y)(x = -1, y = 1) == 0 #test_add_negpos

In [148]:
test_add_standard1(x, y)
test_add_standard2(x, y)
test_add_zeros(x, y)
test_add_negneg(x, y)
test_add_negpos(x, y)

- Вычитание

In [149]:
def test_sub_standard1(x, y):
    assert (x - y)(x = 2, y = 1) == 1 #test_sub_standard1

def test_sub_standard2(x, y):
    assert (x - y)(x = 2, y = 5) == -3 #test_sub_standard2

def test_sub_zeros(x, y):
    assert (x - y)(x = 0, y = 0) == 0 #test_sub_zeros

def test_sub_negneg(x, y):
    assert (x - y)(x = -1, y = -1) == 0 #test_sub_negneg

def test_sub_negpos(x, y):
    assert (x - y)(x = -1, y = 1) == -2 #test_sub_negpos

In [150]:
test_sub_standard1(x, y)
test_sub_standard2(x, y)
test_sub_zeros(x, y)
test_sub_negneg(x, y)
test_sub_negpos(x, y)

- Отрициальное число

In [151]:
def test_neg_standard(x):
    assert (-x)(x = 2) == -2 #test_neg_standard

def test_neg_zero(x):
    assert (-x)(x = 0) == 0 #test_neg_zero

def test_neg_neg(x):
    assert (-x)(x = -2) == 2 #test_neg_neg

In [152]:
test_neg_standard(x)
test_neg_zero(x)
test_neg_neg(x)

- Умножение

In [153]:
def test_mul_standard1(x, y):
    assert (x * y)(x = 2, y = 1) == 2 #test_mul_standard1

def test_mul_standard2(x, y):
    assert (x * y)(x = 2, y = 5) == 10 #test_mul_standard2

def test_mul_zeros(x, y):
    assert (x * y)(x = 0, y = 0) == 0 #test_mul_zeros

def test_mul_negneg(x, y):
    assert (x * y)(x = -1, y = -1) == 1 #test_mul_negneg

def test_mul_negpos(x, y):
    assert (x * y)(x = -1, y = 1) == -1 #test_mul_negpos

In [154]:
test_mul_standard1(x, y)
test_mul_standard2(x, y)
test_mul_zeros(x, y)
test_mul_negneg(x, y)
test_mul_negpos(x, y)

- Деление

In [155]:
def test_div_standard1(x, y):
    assert (x / y)(x = 2, y = 2) == 1.0 #test_div_standard1

def test_div_standard2(x, y):
    assert (x / y)(x = 2, y = 5) == 0.4 #test_div_standard2

def test_div_zero(x, y):
    assert (x / y)(x = 0, y = 1) == 0 #test_div_zero

def test_div_negneg(x, y):
    assert (x / y)(x = -1, y = -5) == 0.2 #test_div_negneg

def test_div_negpos(x, y):
    assert (x / y)(x = -5, y = 2) == -2.5 #test_div_negpos

In [156]:
test_div_standard1(x, y)
test_div_standard2(x, y)
test_div_zero(x, y)
test_div_negneg(x, y)
test_div_negpos(x, y)

- Тесты производных для всех пяти функций

In [157]:
def test_d_neg(x):
     assert (-x).d(x)(x = 2) == -1 #test_d_neg

def test_d_add(x, y):
     assert (y + x).d(x)(x = 2) == 1 #test_d_add(x)
     assert (y + x).d(y)(x = 2) == 1 #test_d_add(y)

def test_d_sub(x, y):
     assert (y - x).d(x)(x = 2) == -1 #test_d_sub(x)
     assert (y - x).d(y)(x = 2) == 1 #test_d_sub(y)

def test_d_mul(x, y):
     assert (y * x).d(x)(x = 2, y = 1) == 1 #test_d_mul(x)
     assert (y * x).d(y)(x = 2, y = 1) == 2 #test_d_mul(y)

def test_d_div(x, y):
     assert (y / x).d(x)(x = 2, y = 1) == -0.25 #test_d_div(x)
     assert (y / x).d(y)(x = 2, y = 1) == 0.5 #test_d_div(y)

In [158]:
test_d_neg(x)
test_d_add(x, y)
test_d_sub(x, y)
test_d_mul(x, y)
test_d_div(x, y)

- Тесты метода Ньютона

In [159]:
def test_Newton_standard1():
    x = Var("x")
    f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
    real = 1
    eps=1e-10
    zero = newton_raphson(f, 0.5, eps)
    assert abs(zero - real) < eps #test_Newton_standard1()

def test_Newton_standard2():
    x = Var("x")
    f = Const(-1) * x * x + Const(3) * x + Const(1)
    real = 3.30278
    eps=1e-5
    zero = newton_raphson(f, 4, eps)
    assert abs(zero - real) < eps #test_Newton_standard2()
    
def test_Newton_standard3():
    x = Var("x")
    f = Const(-1) * x * x * x * x * x * x * x * x * x * x + Const(10) * x * x * x * x + Const(-10)
    real = 1.39458
    eps=1e-3
    zero = newton_raphson(f, 4, eps)
    assert abs(zero - real) < eps #test_Newton_standard3()

In [160]:
test_Newton_standard1()
test_Newton_standard2()
test_Newton_standard3()

- Запустим все тесты вместе

In [161]:
def all_tests(x, y):
    test_add_standard1(x, y)
    test_add_standard2(x, y)
    test_add_zeros(x, y)
    test_add_negneg(x, y)
    test_add_negpos(x, y)

    test_sub_standard1(x, y)
    test_sub_standard2(x, y)
    test_sub_zeros(x, y)
    test_sub_negneg(x, y)
    test_sub_negpos(x, y)

    test_neg_standard(x)
    test_neg_zero(x)
    test_neg_neg(x)

    test_mul_standard1(x, y)
    test_mul_standard2(x, y)
    test_mul_zeros(x, y)
    test_mul_negneg(x, y)
    test_mul_negpos(x, y)

    test_div_standard1(x, y)
    test_div_standard2(x, y)
    test_div_zero(x, y)
    test_div_negneg(x, y)
    test_div_negpos(x, y)

    test_d_neg(x)
    test_d_add(x, y)
    test_d_sub(x, y)
    test_d_mul(x, y)
    test_d_div(x, y)

    test_Newton_standard1()
    test_Newton_standard2()
    test_Newton_standard3()

    print('OK')

    

In [162]:
all_tests(x, y)

OK
